In [2]:
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests

In [154]:
url = "https://www.imdb.com/search/title/?title_type=tv_series&release_date=2000-01-01,2020-12-31&count=100"
response = requests.get(url)

In [155]:
soup = BeautifulSoup(response.text, "html.parser")

In [156]:
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>TV Series,
Released between 2000-01-01 and 2020-12-31
(Sorted by Popularity Ascending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {

In [157]:
tv_containers = soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(tv_containers))
print(len(tv_containers))

<class 'bs4.element.ResultSet'>
100


In [23]:
list(tv_containers)

[<div class="lister-item mode-advanced">
 <div class="lister-top-right">
 <div class="ribbonize" data-caller="filmosearch" data-tconst="tt6468322"></div>
 </div>
 <div class="lister-item-image float-left">
 <a href="/title/tt6468322/"> <img alt="La Casa de Papel" class="loadlate" data-tconst="tt6468322" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BZDcxOGI0MDYtNTc5NS00NDUzLWFkOTItNDIxZjI0OTllNTljXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB466725069_.png" width="67"/>
 </a> </div>
 <div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt6468322/">La Casa de Papel</a>
 <span class="lister-item-year text-muted unbold">(2017– )</span>
 </h3>
 <p class="text-muted">
 <span class="certificate">TV-MA</span>
 <span class="ghost">|</span>
 <span class="runtime">70 min</span>
 <span cl

In [158]:
first_tv = tv_containers[0]
first_tv

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt6468322"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt6468322/"> <img alt="La Casa de Papel" class="loadlate" data-tconst="tt6468322" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BZDcxOGI0MDYtNTc5NS00NDUzLWFkOTItNDIxZjI0OTllNTljXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB466725069_.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt6468322/">La Casa de Papel</a>
<span class="lister-item-year text-muted unbold">(2017– )</span>
</h3>
<p class="text-muted">
<span class="certificate">TV-MA</span>
<span class="ghost">|</span>
<span class="runtime">70 min</span>
<span class="ghost">|</spa

In [27]:
first_tv.div

<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt6468322"></div>
</div>

In [26]:
first_tv.a

<a href="/title/tt6468322/"> <img alt="La Casa de Papel" class="loadlate" data-tconst="tt6468322" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BZDcxOGI0MDYtNTc5NS00NDUzLWFkOTItNDIxZjI0OTllNTljXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB466725069_.png" width="67"/>
</a>

In [28]:
first_tv.h3

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt6468322/">La Casa de Papel</a>
<span class="lister-item-year text-muted unbold">(2017– )</span>
</h3>

In [29]:
first_tv.h3.a

<a href="/title/tt6468322/">La Casa de Papel</a>

In [30]:
first_name = first_tv.h3.a.text
first_name 

'La Casa de Papel'

In [31]:
first_tv.strong

<strong>8.5</strong>

In [32]:
first_imdb = float(first_tv.strong.text)
first_imdb

8.5

In [33]:
first_votes = first_tv.find("span", attrs = {"name": "nv"})
first_votes

<span data-value="216299" name="nv">216,299</span>

In [35]:
first_votes = int(first_votes["data-value"])
first_votes

216299

In [51]:
first_genre = first_tv.find("span", class_ = "genre").text
first_genre

'\nAction, Crime, Mystery            '

In [105]:
final = first_genre.lstrip("\na")
final

'Action, Crime, Mystery            '

In [107]:
final.split()

['Action,', 'Crime,', 'Mystery']

In [73]:
first_run_time = first_tv.find("span", class_ = "runtime")
first_run_time

<span class="runtime">70 min</span>

In [40]:
first_run_time.text

'70 min'

In [44]:
pj = first_tv.find("span", class_ = "runtime").text
pj

'70 min'

In [131]:
first_year = first_tv.h3.find('span', class_ = 'lister-item-year text-muted unbold')
first_year

<span class="lister-item-year text-muted unbold">(2017– )</span>

In [132]:
first_year.text

'(2017– )'

In [115]:
x = None

In [117]:
x.text.split(" ")[0]

AttributeError: 'NoneType' object has no attribute 'text'

In [129]:
first_tv.find("span", class_ = "genre").text.lstrip("\na").strip(' ').split(', ')

['Action', 'Crime', 'Mystery']

In [148]:
all_shows=[]

for tv in tv_containers:
    print(tv.h3.a.text)
    show = {}
    show["name"] = tv.h3.a.text
    show["imdb"] = float(tv.strong.text)
    show["vote"] = int(tv.find("span", attrs = {"name": "nv"})["data-value"])
#     show["run_time"]= int(tv.find("span", class_ = "runtime").text.split(" ")[0]):
    show["genre"] = tv.find("span", class_ = "genre").text.lstrip("\na").strip(' ').split(', ')
    all_shows.append(show)

La Casa de Papel
Ozark
Westworld
The Walking Dead
Community
Better Call Saul
Game of Thrones
Tales from the Loop
Breaking Bad
Grey's Anatomy
Homeland
The Office
Schitt's Creek
Peaky Blinders
Modern Family
Outlander
The Mandalorian
Vikings
Homecoming
Brooklyn Nine-Nine
Supernatural
All American
Stranger Things
The Blacklist
Killing Eve
Friends
Star Trek: Picard
Black Mirror
Lucifer
Criminal Minds
Panchayat
Elite
The Witcher
Rick and Morty
The Wire
Home Before Dark
The Big Show Show
The Sopranos
Dark
The Flash
Sex Education
Hunters
Fargo
Prison Break
Future Man
Hawaii Five-0
The Outsider
Law & Order: Special Victims Unit
Altered Carbon
The Good Doctor
Suits
The Handmaid's Tale
Legacies
Freud
How to Get Away with Murder
Star Wars: The Clone Wars
Shameless
Locke & Key
American Horror Story
Mad Men
The Crown
NCIS
The 100
Zoey's Extraordinary Playlist
The Rookie
This Is Us
The Big Bang Theory
True Detective
Sherlock
You
The Simpsons
Lost
The Boys
Asur: Welcome to Your Dark Side
Narcos
The Va

In [159]:
all_shows=[]
for tv in tv_containers:
#     print(tv.h3.a.text)
    show = {}
    if tv.find("span", class_ = "runtime") is not None:
        show["run_time"]= int(tv.find("span", class_ = "runtime").text.split(" ")[0])
        show["name"] = tv.h3.a.text
        show["imdb"] = float(tv.strong.text)
        show["vote"] = int(tv.find("span", attrs = {"name": "nv"})["data-value"])
        show["year"] = tv.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
        show["genre"] = tv.find("span", class_ = "genre").text.lstrip("\na").strip(' ').split(', ')
    all_shows.append(show)

In [160]:
test_df = pd.DataFrame(all_shows)
print(test_df.info)
test_df

<bound method DataFrame.info of     run_time                       name  imdb      vote         year  \
0       70.0           La Casa de Papel   8.5  216451.0     (2017– )   
1       60.0                      Ozark   8.4  137421.0     (2017– )   
2       62.0                  Westworld   8.7  388614.0     (2016– )   
3       44.0           The Walking Dead   8.2  822003.0     (2010– )   
4       22.0                  Community   8.5  185353.0  (2009–2015)   
..       ...                        ...   ...       ...          ...   
95      60.0           The Last Kingdom   8.3   61769.0     (2015– )   
96      57.0              World on Fire   7.1    2499.0     (2019– )   
97       NaN                        NaN   NaN       NaN          NaN   
98      57.0  The Marvelous Mrs. Maisel   8.7   65834.0     (2017– )   
99       NaN                        NaN   NaN       NaN          NaN   

                        genre  
0    [Action, Crime, Mystery]  
1    [Crime, Drama, Thriller]  
2    [D

,run_time,name,imdb,vote,year,genre
0,70.0,La Casa de Papel,8.5,216451.0,(2017– ),"[Action, Crime, Mystery]"
1,60.0,Ozark,8.4,137421.0,(2017– ),"[Crime, Drama, Thriller]"
2,62.0,Westworld,8.7,388614.0,(2016– ),"[Drama, Mystery, Sci-Fi]"
3,44.0,The Walking Dead,8.2,822003.0,(2010– ),"[Drama, Horror, Thriller]"
4,22.0,Community,8.5,185353.0,(2009–2015),[Comedy]
...,...,...,...,...,...,...
95,60.0,The Last Kingdom,8.3,61769.0,(2015– ),"[Action, Drama, History]"
96,57.0,World on Fire,7.1,2499.0,(2019– ),"[Drama, War]"
97,NaN,NaN,NaN,NaN,NaN,NaN
98,57.0,The Marvelous Mrs. Maisel,8.7,65834.0,(2017– ),"[Comedy, Drama]"


In [187]:
pages = [str(i) for i in range(101,5002,100)]
# years_url = [str(i) for i in range(2000,2018)]

In [190]:
len(pages)

50

In [180]:
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests
from time import sleep
from random import randint
from tqdm import tqdm
# for i in tqdm(range(5)):

In [193]:
all_shows=[]
for page in tqdm(pages):
    response2 = requests.get("https://www.imdb.com/search/title/?title_type=tv_series&release_date=2000-01-01,2019-12-31&count=100&start=" + page + "&ref_=adv_nxt")
    sleep(randint(2,4))
    page_html = BeautifulSoup(response2.text, 'html.parser')
    tv_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')
    for tv in tv_containers:
#     print(tv.h3.a.text)
        show = {}
        if tv.find("span", class_ = "runtime") is not None:
            if tv.strong is not None:
                show["run_time"]= int(tv.find("span", class_ = "runtime").text.split(" ")[0])
                show["name"] = tv.h3.a.text
                show["imdb"] = float(tv.strong.text)
                show["vote"] = int(tv.find("span", attrs = {"name": "nv"})["data-value"])
                show["year"] = tv.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
                show["genre"] = tv.find("span", class_ = "genre").text.lstrip("\na").strip(' ').split(', ')
            all_shows.append(show)



  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:05<04:07,  5.06s/it]

  4%|▍         | 2/50 [00:09<03:50,  4.80s/it]

  6%|▌         | 3/50 [00:14<03:55,  5.00s/it]

  8%|▊         | 4/50 [00:20<03:55,  5.12s/it]

 10%|█         | 5/50 [00:23<03:24,  4.53s/it]

 12%|█▏        | 6/50 [00:26<03:00,  4.09s/it]

 14%|█▍        | 7/50 [00:30<03:00,  4.21s/it]

 16%|█▌        | 8/50 [00:34<02:47,  3.98s/it]

 18%|█▊        | 9/50 [00:37<02:34,  3.78s/it]

 20%|██        | 10/50 [00:42<02:48,  4.22s/it]

 22%|██▏       | 11/50 [00:48<03:02,  4.69s/it]

 24%|██▍       | 12/50 [00:53<02:57,  4.67s/it]

 26%|██▌       | 13/50 [00:58<03:01,  4.89s/it]

 28%|██▊       | 14/50 [01:04<03:01,  5.05s/it]

 30%|███       | 15/50 [01:09<03:01,  5.18s/it]

 32%|███▏      | 16/50 [01:15<03:02,  5.38s/it]

 34%|███▍      | 17/50 [01:20<02:59,  5.44s/it]

 36%|███▌      | 18/50 [01:26<02:55,  5.50s/it]

 38%|███▊      | 19/50 [01:30<02:31,  4.88s/it]

 40%|████      | 20/50 [01:34<02:24,

In [195]:
len(all_shows)

4250

In [199]:
df = pd.DataFrame(all_shows)
df

,run_time,name,imdb,vote,year,genre
0,40.0,Bones,7.8,133688.0,(2005–2017),"[Comedy, Crime, Drama]"
1,58.0,Downton Abbey,8.7,152777.0,(2010–2015),"[Drama, Romance]"
2,43.0,Blue Bloods,7.5,30599.0,(2010– ),"[Crime, Drama, Mystery]"
3,22.0,The Good Place,8.2,92511.0,(2016–2020),"[Comedy, Drama, Fantasy]"
4,44.0,Gilmore Girls,8.1,91945.0,(2000–2007),"[Comedy, Drama]"
...,...,...,...,...,...,...
4245,60.0,Game of Silence,7.0,2400.0,(2016),[Drama]
4246,23.0,The Land Before Time,6.4,594.0,(2007– ),"[Animation, Short, Adventure]"
4247,42.0,My Big Fat Fabulous Life,3.7,730.0,(2015– ),[Reality-TV]
4248,50.0,Celebrity Gogglebox,7.0,53.0,(2019– ),[Reality-TV]


In [204]:
f_df = pd.concat([test_df, df])
f_df.head(20)

,run_time,name,imdb,vote,year,genre
0,70.0,La Casa de Papel,8.5,216451.0,(2017– ),"[Action, Crime, Mystery]"
1,60.0,Ozark,8.4,137421.0,(2017– ),"[Crime, Drama, Thriller]"
2,62.0,Westworld,8.7,388614.0,(2016– ),"[Drama, Mystery, Sci-Fi]"
3,44.0,The Walking Dead,8.2,822003.0,(2010– ),"[Drama, Horror, Thriller]"
4,22.0,Community,8.5,185353.0,(2009–2015),[Comedy]
5,46.0,Better Call Saul,8.7,281241.0,(2015– ),"[Crime, Drama]"
6,57.0,Game of Thrones,9.3,1659569.0,(2011–2019),"[Action, Adventure, Drama]"
7,NaN,NaN,NaN,NaN,NaN,NaN
8,49.0,Breaking Bad,9.5,1333210.0,(2008–2013),"[Crime, Drama, Thriller]"
9,41.0,Grey's Anatomy,7.6,231264.0,(2005– ),"[Drama, Romance]"


In [205]:
f_df.to_csv("Final_DataFrame")

In [208]:
f_df = f_df.dropna(axis = 0)
f_df.head(20)

,run_time,name,imdb,vote,year,genre
0,70.0,La Casa de Papel,8.5,216451.0,(2017– ),"[Action, Crime, Mystery]"
1,60.0,Ozark,8.4,137421.0,(2017– ),"[Crime, Drama, Thriller]"
2,62.0,Westworld,8.7,388614.0,(2016– ),"[Drama, Mystery, Sci-Fi]"
3,44.0,The Walking Dead,8.2,822003.0,(2010– ),"[Drama, Horror, Thriller]"
4,22.0,Community,8.5,185353.0,(2009–2015),[Comedy]
5,46.0,Better Call Saul,8.7,281241.0,(2015– ),"[Crime, Drama]"
6,57.0,Game of Thrones,9.3,1659569.0,(2011–2019),"[Action, Adventure, Drama]"
8,49.0,Breaking Bad,9.5,1333210.0,(2008–2013),"[Crime, Drama, Thriller]"
9,41.0,Grey's Anatomy,7.6,231264.0,(2005– ),"[Drama, Romance]"
10,55.0,Homeland,8.3,293969.0,(2011–2020),"[Crime, Drama, Mystery]"


In [209]:
f_df = f_df.reset_index(drop = True)
f_df.head(20)

,run_time,name,imdb,vote,year,genre
0,70.0,La Casa de Papel,8.5,216451.0,(2017– ),"[Action, Crime, Mystery]"
1,60.0,Ozark,8.4,137421.0,(2017– ),"[Crime, Drama, Thriller]"
2,62.0,Westworld,8.7,388614.0,(2016– ),"[Drama, Mystery, Sci-Fi]"
3,44.0,The Walking Dead,8.2,822003.0,(2010– ),"[Drama, Horror, Thriller]"
4,22.0,Community,8.5,185353.0,(2009–2015),[Comedy]
5,46.0,Better Call Saul,8.7,281241.0,(2015– ),"[Crime, Drama]"
6,57.0,Game of Thrones,9.3,1659569.0,(2011–2019),"[Action, Adventure, Drama]"
7,49.0,Breaking Bad,9.5,1333210.0,(2008–2013),"[Crime, Drama, Thriller]"
8,41.0,Grey's Anatomy,7.6,231264.0,(2005– ),"[Drama, Romance]"
9,55.0,Homeland,8.3,293969.0,(2011–2020),"[Crime, Drama, Mystery]"


In [210]:
r = []
for i in f_df['genre'].values:
    for x in i:
        r.append(x)
r

['Action',
 'Crime',
 'Mystery',
 'Crime',
 'Drama',
 'Thriller',
 'Drama',
 'Mystery',
 'Sci-Fi',
 'Drama',
 'Horror',
 'Thriller',
 'Comedy',
 'Crime',
 'Drama',
 'Action',
 'Adventure',
 'Drama',
 'Crime',
 'Drama',
 'Thriller',
 'Drama',
 'Romance',
 'Crime',
 'Drama',
 'Mystery',
 'Comedy',
 'Comedy',
 'Crime',
 'Drama',
 'Comedy',
 'Drama',
 'Romance',
 'Drama',
 'Fantasy',
 'Romance',
 'Action',
 'Adventure',
 'Sci-Fi',
 'Action',
 'Adventure',
 'Drama',
 'Drama',
 'Thriller',
 'Comedy',
 'Crime',
 'Drama',
 'Fantasy',
 'Horror',
 'Drama',
 'Sport',
 'Drama',
 'Fantasy',
 'Horror',
 'Crime',
 'Drama',
 'Mystery',
 'Action',
 'Adventure',
 'Drama',
 'Action',
 'Adventure',
 'Drama',
 'Drama',
 'Sci-Fi',
 'Thriller',
 'Crime',
 'Drama',
 'Fantasy',
 'Crime',
 'Drama',
 'Mystery',
 'Crime',
 'Drama',
 'Thriller',
 'Action',
 'Adventure',
 'Drama',
 'Animation',
 'Adventure',
 'Comedy',
 'Crime',
 'Drama',
 'Thriller',
 'Crime',
 'Drama',
 'Mystery',
 'Action',
 'Adventure',
 'Drama

In [214]:
r = list(set(r))
r

['War',
 'Action',
 'Game-Show',
 'Fantasy',
 'Adventure',
 'Short',
 'Mystery',
 'Western',
 'Animation',
 'Talk-Show',
 'Documentary',
 'Drama',
 'Thriller',
 'Musical',
 'Sci-Fi',
 'Comedy',
 'Family',
 'News',
 'Biography',
 'History',
 'Crime',
 'Reality-TV',
 'Music',
 'Romance',
 'Sport',
 'Horror']

In [217]:
for i in r:
    f_df[i] = 0

In [219]:
f_df.head(20)

,run_time,name,imdb,vote,year,genre,War,Action,Game-Show,Fantasy,...,Family,News,Biography,History,Crime,Reality-TV,Music,Romance,Sport,Horror
0,70.0,La Casa de Papel,8.5,216451.0,(2017– ),"[Action, Crime, Mystery]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,60.0,Ozark,8.4,137421.0,(2017– ),"[Crime, Drama, Thriller]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62.0,Westworld,8.7,388614.0,(2016– ),"[Drama, Mystery, Sci-Fi]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,44.0,The Walking Dead,8.2,822003.0,(2010– ),"[Drama, Horror, Thriller]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,22.0,Community,8.5,185353.0,(2009–2015),[Comedy],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,46.0,Better Call Saul,8.7,281241.0,(2015– ),"[Crime, Drama]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,57.0,Game of Thrones,9.3,1659569.0,(2011–2019),"[Action, Adventure, Drama]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,49.0,Breaking Bad,9.5,1333210.0,(2008–2013),"[Crime, Drama, Thriller]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,41.0,Grey's Anatomy,7.6,231264.0,(2005– ),"[Drama, Romance]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,55.0,Homeland,8.3,293969.0,(2011–2020),"[Crime, Drama, Mystery]",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [231]:
def labelGenres(row):
    if len(row['genre']) > 0:
        for genre in row['genre']:
            row[genre] += 2
    return row

In [232]:
f_df = f_df.apply(labelGenres, axis = 1)
f_df.head(20)

,run_time,name,imdb,vote,year,genre,War,Action,Game-Show,Fantasy,...,Family,News,Biography,History,Crime,Reality-TV,Music,Romance,Sport,Horror
0,70.0,La Casa de Papel,8.5,216451.0,(2017– ),"[Action, Crime, Mystery]",0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
1,60.0,Ozark,8.4,137421.0,(2017– ),"[Crime, Drama, Thriller]",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
2,62.0,Westworld,8.7,388614.0,(2016– ),"[Drama, Mystery, Sci-Fi]",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,44.0,The Walking Dead,8.2,822003.0,(2010– ),"[Drama, Horror, Thriller]",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
4,22.0,Community,8.5,185353.0,(2009–2015),[Comedy],0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,46.0,Better Call Saul,8.7,281241.0,(2015– ),"[Crime, Drama]",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
6,57.0,Game of Thrones,9.3,1659569.0,(2011–2019),"[Action, Adventure, Drama]",0.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,49.0,Breaking Bad,9.5,1333210.0,(2008–2013),"[Crime, Drama, Thriller]",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0
8,41.0,Grey's Anatomy,7.6,231264.0,(2005– ),"[Drama, Romance]",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
9,55.0,Homeland,8.3,293969.0,(2011–2020),"[Crime, Drama, Mystery]",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0


In [233]:
f_df.to_csv("Basically_Done.csv")

In [ ]:
zero_dict = {88:0, 
            77: np.nan,
            99: np.nan}

f_df['PHYSHLTH'] = f_df['genre'].replace(zero_dict)

In [ ]:
pd.Series({0: ['a', 'b', 'c'], 1:['c'], 2: ['b', 'c', 'e'], 3: ['a', 'c'],

In [ ]:
pd.get_dummies(s.apply(pd.Series).stack()).sum(level=0)

In [63]:
# tv_containers[0]

In [58]:
# Lists to store the scraped data in
names = []
imdb_ratings = []
genres = []
votes = []
run_time = []
#Extract data from individual movie container
for tv in tv_containers[0:1]:
    #print(tv)
    
#     name = tv.h3.a.text
    
#     names.append(name)
    
#     imdb = float(tv.strong.text)
#     imdb_ratings.append(imdb)
    
#     vote = tv.find("span", attrs = {"name": "nv"})["data-value"]
#     votes.append(int(vote))
    
#     rt = tv.find("span", class_ = "runtime")
#     print(rt.text)
#     run_time.append(rt)
    
#     genre = tv.find("span", class_ = "genre").text
#     genres.append(genre)
    


70 min


In [61]:
run_time[0].text

'70 min'

In [66]:
run_time[0].text.split(" ")

['70', 'min']

In [112]:
int(run_time[0].text.split(" ")[0])

70

In [62]:
genre

'\nAction, Crime, Mystery            '

In [50]:
test_df = pd.DataFrame({"TV": names, "Ratings": imdb_ratings, "Genres": genres, "Total Votes": votes, "Run Time": run_time})
print(test_df.info)
test_df

<bound method DataFrame.info of                                 TV  Ratings  \
0                 La Casa de Papel      8.5   
1                            Ozark      8.4   
2                        Westworld      8.7   
3                 The Walking Dead      8.2   
4                        Community      8.5   
..                             ...      ...   
95                      Mindhunter      8.6   
96                          Dexter      8.6   
97            Star Trek: Discovery      7.3   
98                      Doctor Who      8.6   
99  Star Trek: The Next Generation      8.6   

                                      Genres  Total Votes  Run Time  
0       \nAction, Crime, Mystery                   216299  [70 min]  
1       \nCrime, Drama, Thriller                   137351  [60 min]  
2       \nDrama, Mystery, Sci-Fi                   388573  [62 min]  
3      \nDrama, Horror, Thriller                   821985  [44 min]  
4                       \nComedy                   18

,TV,Ratings,Genres,Total Votes,Run Time
0,La Casa de Papel,8.5,"\nAction, Crime, Mystery",216299,[70 min]
1,Ozark,8.4,"\nCrime, Drama, Thriller",137351,[60 min]
2,Westworld,8.7,"\nDrama, Mystery, Sci-Fi",388573,[62 min]
3,The Walking Dead,8.2,"\nDrama, Horror, Thriller",821985,[44 min]
4,Community,8.5,\nComedy,185322,[22 min]
...,...,...,...,...,...
95,Mindhunter,8.6,"\nCrime, Drama, Thriller",183548,[60 min]
96,Dexter,8.6,"\nCrime, Drama, Mystery",619761,[53 min]
97,Star Trek: Discovery,7.3,"\nAction, Adventure, Drama",78975,[60 min]
98,Doctor Who,8.6,"\nAdventure, Drama, Family",192856,[45 min]


In [ ]:
<a, <strong, <span, <h3,  

In [27]:
for child in soup.h3.children:
    print(child)



<span class="lister-item-index unbold text-primary">1.</span>


<a href="/title/tt6468322/">La Casa de Papel</a>


<span class="lister-item-year text-muted unbold">(2017– )</span>




In [85]:
test = soup.find_all("h3")
test[0].getText()

'\n1.\nLa Casa de Papel\n(2017– )\n'

In [89]:
test

[<h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt6468322/">La Casa de Papel</a>
 <span class="lister-item-year text-muted unbold">(2017– )</span>
 </h3>, <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">2.</span>
 <a href="/title/tt5071412/">Ozark</a>
 <span class="lister-item-year text-muted unbold">(2017– )</span>
 </h3>, <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">3.</span>
 <a href="/title/tt0475784/">Westworld</a>
 <span class="lister-item-year text-muted unbold">(2016– )</span>
 </h3>, <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">4.</span>
 <a href="/title/tt1520211/">The Walking Dead</a>
 <span class="lister-item-year text-muted unbold">(2010– )</span>
 </h3>, <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">5.</span>
 <a href="/title/tt1439629/">Community</a>
 <s

In [88]:
ip = soup.find_all("h3", class_ = "lister-item-header")
ip

[<h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt6468322/">La Casa de Papel</a>
 <span class="lister-item-year text-muted unbold">(2017– )</span>
 </h3>, <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">2.</span>
 <a href="/title/tt5071412/">Ozark</a>
 <span class="lister-item-year text-muted unbold">(2017– )</span>
 </h3>, <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">3.</span>
 <a href="/title/tt0475784/">Westworld</a>
 <span class="lister-item-year text-muted unbold">(2016– )</span>
 </h3>, <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">4.</span>
 <a href="/title/tt1520211/">The Walking Dead</a>
 <span class="lister-item-year text-muted unbold">(2010– )</span>
 </h3>, <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">5.</span>
 <a href="/title/tt1439629/">Community</a>
 <s

In [90]:
for v in ip:
    print(v)

<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt6468322/">La Casa de Papel</a>
<span class="lister-item-year text-muted unbold">(2017– )</span>
</h3>
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">2.</span>
<a href="/title/tt5071412/">Ozark</a>
<span class="lister-item-year text-muted unbold">(2017– )</span>
</h3>
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">3.</span>
<a href="/title/tt0475784/">Westworld</a>
<span class="lister-item-year text-muted unbold">(2016– )</span>
</h3>
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">4.</span>
<a href="/title/tt1520211/">The Walking Dead</a>
<span class="lister-item-year text-muted unbold">(2010– )</span>
</h3>
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">5.</span>
<a href="/title/tt1439629/">Community</a>
<span class="lister-item-y

In [63]:
hi = list(soup.children)[1]
# hi

In [69]:
list(hi.children)

AttributeError: 'Doctype' object has no attribute 'children'

In [ ]:
[type(item) for item in list(soup.children)]

In [52]:
soup.prettify()

'<!DOCTYPE html>\n<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>\n  <script type="text/javascript">\n   var IMDbTimer={starttime: new Date().getTime(),pt:\'java\'};\n  </script>\n  <script>\n   if (typeof uet == \'function\') {\n      uet("bb", "LoadTitle", {wb: 1});\n    }\n  </script>\n  <script>\n   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);\n  </script>\n  <title>\n   TV Series\n(Sorted by Popularity Ascending) - IMDb\n  </title>\n  <script>\n   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);\n  </script>\n  <script>\n   if (typeof uet == \'function\') {\n      uet("be", "LoadTitle", {wb: 1});\n    }\n  </script>\n  <script>\n   if (typeof uex == \'f

In [64]:
[type(item) for item in list(soup.children)]

[bs4.element.NavigableString,
 bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [81]:
html = list(soup.children)[3]
html

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>TV Series
(Sorted by Popularity Ascending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/sear